# Import Prerequisite Libraries

In [28]:
from Funcs.Setup import *
import numpy as np
import pandas as pd

# import statsmodels.api as sm
from sklearn.feature_selection import VarianceThreshold
# from ISLP import confusion_table
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier 
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from datetime import timedelta as td
from sklearn.model_selection import GroupKFold
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn.impute import KNNImputer
import warnings
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LassoCV
import shap
import matplotlib.pyplot as plt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import optuna
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
warnings.filterwarnings("ignore")


# Load Dataset

In [29]:
RANDOM_STATE = 42

In [30]:
def user_demographic(demo):
    demo = pd.get_dummies(demo, columns=['Gender'])
    selected_columns = ['UID', 'Extroversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness', 'PHQ-9', 'GAD-7', 'PSS', 'GHQ', 'Gender_F', 'Gender_M']
    demo = demo[selected_columns]
    demo = demo.rename(columns={'UID': 'uid'})  # rename UID to uid
    return demo

In [31]:
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 1 #Morning
    elif 12 <= hour < 17:
        return 2 #Afternoon
    elif 17 <= hour < 21:
        return 3 #Evening
    else:
        return 4 #Night

In [ ]:
def load_aqara_data_updated(file_name, hour, yesterday):
    aqara_data = pd.read_csv(file_name, index_col=None)
            
    suffix = f'_{hour*60}min'

    filtered_columns = [col for col in aqara_data.columns if col.endswith(suffix)]
     
    # Select the filtered columns along with any essential columns like 'uid', 'startTime'
    essential_columns = ['uid', 'startTime']
    aqara_features = aqara_data[essential_columns + filtered_columns]

    if yesterday == True:
        # 각 행에서 '[None]' 인 열이 하나라도 있으면, 그 행 제거
        aqara_features = aqara_features.replace(np.nan, 0)
        aqara_features = aqara_features.replace('[None]', np.nan)

    # Identify aqara columns only (after filtering by hour)
    aqara_columns_only = [col for col in aqara_features.columns if col.startswith('aqara_')]

    for col in aqara_columns_only:
        # Convert the column to numeric, setting errors='coerce' will turn invalid parsing into NaN
        aqara_features[col] = pd.to_numeric(aqara_features[col], errors='coerce')
        
        # Calculate the mean per uid
        mean_per_uid = aqara_features.groupby('uid')[col].transform('mean')
        
        # Calculate variance and comparison
        aqara_features[f'{col}_deviation'] = aqara_features[col] - mean_per_uid
        aqara_features[f'{col}_comparison'] = (aqara_features[col] > mean_per_uid).astype(int) - (aqara_features[col] < mean_per_uid).astype(int)

    # Calculate routines based on filtered columns
    if yesterday == False:
        fridge_columns = [col for col in aqara_columns_only if 'fridge_ImmediatePast' in col]
        microwave_columns = [col for col in aqara_columns_only if 'microwave_ImmediatePast' in col]

        cleaner_columns = [col for col in aqara_columns_only if 'cleaner_ImmediatePast' in col]
        washer_columns = [col for col in aqara_columns_only if 'washer_ImmediatePast' in col]
    
        if fridge_columns and microwave_columns:
            aqara_features[f'aqara_eating_routine_immediatePast_{hour*60}min'] = (
                (aqara_features[fridge_columns[0]] >= 1) | 
                (aqara_features[microwave_columns[0]] >= 1)
            ).astype(int)
            
        if cleaner_columns and washer_columns:
            aqara_features[f'aqara_chores_routine_immediatePast_{hour*60}min'] = (
                (aqara_features[cleaner_columns[0]] >= 1) | 
                (aqara_features[washer_columns[0]] >= 1)
            ).astype(int)

    else:
        fridge_columns = [col for col in aqara_columns_only if 'fridge_yesterday' in col and 'mean' in col]
        microwave_columns = [col for col in aqara_columns_only if 'microwave_yesterday' in col and 'mean' in col]

        cleaner_columns = [col for col in aqara_columns_only if 'cleaner_yesterday' in col and 'mean' in col]
        washer_columns = [col for col in aqara_columns_only if 'washer_yesterday' in col and 'mean' in col]

        # 공통 시간대 접미사를 찾고 새로운 열을 생성
        for fridge_col in fridge_columns:
            for microwave_col in microwave_columns:
                # 공통 시간대 접미사를 확인
                if fridge_col.split('fridge_yesterday_')[1] == microwave_col.split('microwave_yesterday_')[1]:
                    common_suffix = fridge_col.split('fridge_yesterday_')[1]  # 공통 접미사 추출
                    new_col_name = f'aqara_eating_routine_yesterday_{common_suffix}'

                    # 새로운 열을 생성하고 값을 설정
                    aqara_features[new_col_name] = (
                        (aqara_features[fridge_col] >= 1) | 
                        (aqara_features[microwave_col] >= 1)
                    ).astype(int)

        # 공통 시간대 접미사를 찾고 새로운 열을 생성 (chores routine)
        for cleaner_col in cleaner_columns:
            for washer_col in washer_columns:
                # 공통 시간대 접미사를 확인
                if cleaner_col.split('cleaner_yesterday_')[1] == washer_col.split('washer_yesterday_')[1]:
                    common_suffix = cleaner_col.split('cleaner_yesterday_')[1]  # 공통 접미사 추출
                    new_col_name = f'aqara_chores_routine_yesterday_{common_suffix}'

                    # 새로운 열을 생성하고 값을 설정
                    aqara_features[new_col_name] = (
                        (aqara_features[cleaner_col] >= 1) | 
                        (aqara_features[washer_col] >= 1)
                    ).astype(int)


    return aqara_features


In [ ]:
def load_smartphone_data(file_name):
    smartphone = load(file_name)
    smartphone_feature, _, uids, dates = smartphone

    smartphone_feature = smartphone_feature.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x.astype('float64'), axis=0)

    df_smartphone = pd.concat([smartphone_feature, pd.DataFrame(uids.tolist(), columns=['uid']), pd.DataFrame(dates.tolist(), columns=['timestamp'])], axis=1)
    df_smartphone['uid'] = df_smartphone['uid'].str.replace('P', '').astype('int64')
    df_smartphone['hour'] = df_smartphone['timestamp'].dt.hour
    df_smartphone['part_of_day'] = df_smartphone['hour'].apply(get_part_of_day)
    df_smartphone['timestamp'] = df_smartphone['timestamp'].apply(to_unix_timestamp)
        

    # df_smartphone 의 LOC_LABEL#RLV_SUP=eating#ImmediatePast_60,LOC_LABEL#RLV_SUP=home#ImmediatePast_60,LOC_LABEL#RLV_SUP=work#ImmediatePast_60,LOC_LABEL#RLV_SUP=social#ImmediatePast_60,LOC_LABEL#RLV_SUP=others#ImmediatePast_60 열 중 LOC_LABEL#RLV_SUP=eating#ImmediatePast_60 이 가장 큰 값을 가지는 경우 1, 아니면 0
    df_smartphone['eating_routine_immediatePast_60'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#ImmediatePast_60'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#ImmediatePast_60', 
                'LOC_LABEL#RLV_SUP=home#ImmediatePast_60', 
                'LOC_LABEL#RLV_SUP=work#ImmediatePast_60', 
                'LOC_LABEL#RLV_SUP=social#ImmediatePast_60', 
                'LOC_LABEL#RLV_SUP=others#ImmediatePast_60'
            ]
        ].max(axis=1)
        ).astype(int)

    df_smartphone['eating_routine_YesterdayDawn'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#YesterdayDawn'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#YesterdayDawn', 
                'LOC_LABEL#RLV_SUP=home#YesterdayDawn', 
                'LOC_LABEL#RLV_SUP=work#YesterdayDawn', 
                'LOC_LABEL#RLV_SUP=social#YesterdayDawn', 
                'LOC_LABEL#RLV_SUP=others#YesterdayDawn'
            ]
        ].max(axis=1)
        ).astype(int)

    df_smartphone['eating_routine_YesterdayMorning'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#YesterdayMorning'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#YesterdayMorning', 
                'LOC_LABEL#RLV_SUP=home#YesterdayMorning', 
                'LOC_LABEL#RLV_SUP=work#YesterdayMorning', 
                'LOC_LABEL#RLV_SUP=social#YesterdayMorning', 
                'LOC_LABEL#RLV_SUP=others#YesterdayMorning'
            ]
        ].max(axis=1)
        ).astype(int)

    df_smartphone['eating_routine_YesterdayAfternoon'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#YesterdayAfternoon'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#YesterdayAfternoon', 
                'LOC_LABEL#RLV_SUP=home#YesterdayAfternoon', 
                'LOC_LABEL#RLV_SUP=work#YesterdayAfternoon', 
                'LOC_LABEL#RLV_SUP=social#YesterdayAfternoon', 
                'LOC_LABEL#RLV_SUP=others#YesterdayAfternoon'
            ]
        ].max(axis=1)
        ).astype(int)
    

    df_smartphone['eating_routine_YesterdayEvening'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#YesterdayEvening'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#YesterdayEvening', 
                'LOC_LABEL#RLV_SUP=home#YesterdayEvening', 
                'LOC_LABEL#RLV_SUP=work#YesterdayEvening', 
                'LOC_LABEL#RLV_SUP=social#YesterdayEvening', 
                'LOC_LABEL#RLV_SUP=others#YesterdayEvening'
            ]
        ].max(axis=1)
        ).astype(int)
    
    df_smartphone['eating_routine_YesterdayNight'] = (
        df_smartphone['LOC_LABEL#RLV_SUP=eating#YesterdayNight'] == df_smartphone[
            [
                'LOC_LABEL#RLV_SUP=eating#YesterdayNight', 
                'LOC_LABEL#RLV_SUP=home#YesterdayNight', 
                'LOC_LABEL#RLV_SUP=work#YesterdayNight', 
                'LOC_LABEL#RLV_SUP=social#YesterdayNight', 
                'LOC_LABEL#RLV_SUP=others#YesterdayNight'
            ]
        ].max(axis=1)
        ).astype(int)
    
    # Today Epoch feature removed
    df_smartphone = df_smartphone[df_smartphone.columns.drop(list(df_smartphone.filter(regex='_Today')))]

    return df_smartphone

In [34]:
def load_label_data(file_name):

    label = pd.read_csv(file_name, index_col=None)
    label['phq4_result'] = label['phq2_result'] + label['gad2_result']
    df_label = binarize_by_range(label)

    return label

In [35]:
df_label = load_label_data('FEATURES/label_2023.csv')
df_audio = pd.read_pickle('FEATURES/librosa_modified.pkl')
df_speech = pd.read_csv('FEATURES/speech_data_2023.csv', index_col=None) 
df_bluSensor = pd.read_csv('FEATURES/bluSensor_features_15min.csv', index_col=None)
df_fitbit = pd.read_csv('FEATURES/fitbit_features_24h.csv', index_col=None)
# df_aqara = load_aqara_data('FEATURES/aqara_features_12h_24h_without_env.csv')
df_aqara_before = load_aqara_data_updated('FEATURES/aqara_before_1h_3h_6h_12h_and.csv', 1, yesterday=False)
df_aqara_yesterday = load_aqara_data_updated('FEATURES/aqara_yesterday_12h.csv', 12, yesterday=True)

df_withings = pd.read_csv('FEATURES/withings_features_24h.csv', index_col=None)
df_pre_survey = pd.read_csv('FEATURES/user_demographics_pre_test_2023.csv', index_col=None)
df_smartphone = load_smartphone_data('FEATURES/smartphone_features_60min_yesterday_today_impute_median.pkl')
df_demo = user_demographic(df_pre_survey)

df_label.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_audio.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_aqara_before.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_aqara_yesterday.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_withings.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_fitbit.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_demo.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_speech.rename(columns={'startTime': 'timestamp'}, inplace=True)
df_bluSensor.rename(columns={'startTime': 'timestamp'}, inplace=True)




In [36]:
len(df_label)

1619

# Use Data Combination

In [ ]:
data_sources = [
    (df_smartphone, ['uid', 'timestamp'], None, None),   # Smartphone 데이터
    (df_audio, ['uid', 'timestamp'], lambda df: df['duration'] > 0, None), # Audio 데이터, 조건 적용
    (df_aqara_before, ['uid', 'timestamp'], None, 0),       # aqara 센서 데이터
    (df_aqara_yesterday, ['uid', 'timestamp'], None, 0),       # aqara 센서 데이터
    (df_withings, ['uid', 'timestamp'], None, 0),    # withings 센서 데이터
    (df_fitbit, ['uid', 'timestamp'], None, 0),   # Fitbit 데이터
    # (df_demo, ['uid'], None, None),                  # 데모그래픽 데이터
    (df_speech, ['uid', 'timestamp'], None, None),
    (df_bluSensor, ['uid', 'timestamp'], None, None), # 블루센서 데이터 
]

In [38]:
def data_sources_combinations(base_df, data_sources):
    merged_df = base_df.copy()
    for source_df, on_columns, filter_condition, fill_na in data_sources:
        merged_df = pd.merge(merged_df, source_df, on=on_columns, how='left')
       
        if filter_condition is not None:
            merged_df = merged_df.loc[filter_condition(merged_df)]

        if fill_na is not None:
            merged_df.fillna(fill_na, inplace=True)

    return merged_df

In [39]:
# Drop the user with skewed label distribution
# df_label = df_label.groupby('uid').filter(lambda x: (x[label].nunique() > 1))
base_df = df_label.copy()

merged_df = data_sources_combinations(base_df, data_sources)
merged_df.columns = merged_df.columns.map(str)

unnamed_columns = [col for col in merged_df.columns if 'Unnamed' in col]
merged_df = merged_df.drop(columns=unnamed_columns)
merged_df = merged_df.set_index('timestamp').sort_index()

# print(merged_df.shape)
# rows_with_nan = merged_df[merged_df.isna().any(axis=1)]
# print(rows_with_nan.shape)

In [40]:
# Unnamed가 포함된 컬럼 이름이 있는지 확인하는 코드
merged_df.columns = merged_df.columns.map(str)
unnamed_columns = [col for col in merged_df.columns if 'Unnamed' in col]

if unnamed_columns:
    print(f"Unnamed가 포함된 컬럼 이름: {unnamed_columns}")
else:
    print("Unnamed가 포함된 컬럼 이름이 없습니다.")

Unnamed가 포함된 컬럼 이름이 없습니다.


In [ ]:
### ESM에서 음성 데이터까지 완벽하게 있는 데이터만 필터링 
# merged_df: 음성 제외 원하는 source merged_df
merged_df = pd.merge(merged_df, df_audio[df_audio['duration'] > 0], on=['uid', 'timestamp'], how='inner')
all_merged_dropna_df = merged_df.dropna()
merged_df = merged_df.iloc[:, :-182]

In [44]:
# categorical data 컬럼 개수 출력
categorical_data = merged_df.select_dtypes(include=['object'])
print(f"Categorical data 컬럼 개수: {len(categorical_data.columns)}")

# numerical data 컬럼 개수 출력
numerical_data = merged_df.select_dtypes(include=['float64', 'int'])
print(f"Numerical data 컬럼 개수: {len(numerical_data.columns)}")

Categorical data 컬럼 개수: 0
Numerical data 컬럼 개수: 2925


In [45]:
# Missing value 개수 확인
missing_values = merged_df.isnull().sum().sum()
print(f"Missing values: {missing_values}")

Missing values: 30349


In [46]:
len(merged_df)

1157

# Classification Models

In [47]:
def check_single_class_groups(df, label):
    single_class_groups = []
    grouped = df.groupby('uid')

    for group_id, group in grouped:
        if len(group[label].unique()) == 1:
            single_class_groups.append(group_id)
    
    return single_class_groups



In [48]:
def data_splitting(df, label):

    to_drop = ['phq4_result', 'phq4_result_binary', 'phq2_result', 'gad2_result', 'stress_result', 'posNeg_result', 'arousal_result', 'phq2_result_binary', 'gad2_result_binary', 'stress_result_binary', 'posNeg_result_binary', 'arousal_result_binary']

    # define X and y dataset
    X = df.drop(columns=to_drop + ['uid'])
    y = df[label].values
    UIDS = df['uid'].values

    
    return X, y, UIDS

In [49]:
def remove_user_with_skewed_label(data, label):
    # 'phq_result'가 0인 비율 계산
    label_ratios = data[data[label] == 0].groupby('uid').size() / data.groupby('uid').size()
    label_ratios = label_ratios.fillna(0)

    users_to_remove = label_ratios[(label_ratios < 0.1) | (label_ratios > 0.9)].index
    # print(label_ratios)  

    return users_to_remove

In [50]:
def select_features_lasso(data, target):
    """
    Performs feature selection using LASSO regression.
    
    Args:
        data (pd.DataFrame): The input dataset with features.
        target (pd.Series): The target variable.
        
    Returns:
        pd.DataFrame: The dataset with features selected by LASSO.
    """
    lasso = LassoCV(cv=5, random_state=42, alphas=[0.001, 0.01, 0.1, 1, 10]).fit(data, target)
    selected_features = data.columns[lasso.coef_ != 0]
    return selected_features


In [51]:
# 소수 클래스의 크기를 가져오는 함수
def get_minority_class_size(y):
    counter = Counter(y)
    return min(counter.values())

In [ ]:
def train_ml(data, label, model_instance, zero, high_pariwise, lasso, scaling, cv_method='loso', oversample=False, sampling_method=None, correlation_threshold=None):
    """
    Classify data using a given pre-instantiated model with a Leave-One-Subject-Out (LOSO) cross-validation approach
    on the complete data of 5 weeks.

    Args:
    - data: The dataset to classify, expected to span exactly 5 weeks.
    - model_instance: A pre-instantiated model ready for training.
    - zero: Indicates whether to remove features with zero variance.
    - high_pariwise: Indicates whether to remove highly correlated features.
    - oversample: Indicates whether to apply oversampling to the training data.
    - sampling_method: The strategy to use for oversampling if enabled.
    - correlation_threshold: Threshold for removing highly correlated features.
    """

    #interpolate to fill missing values as much as possible
    # data = data.groupby('uid').apply(lambda group: group.interpolate())

    X, y, groups = data_splitting(data, label)
    num_users = np.unique(groups).size
    y = pd.Series(y).astype(float)

 
    # Select cross-validation method
    if cv_method == 'loso':
        cv = LeaveOneGroupOut().split(X, y, groups=groups)
    elif cv_method == 'kfold':
        cv = KFold(n_splits=10, shuffle=False, random_state=RANDOM_STATE).split(X, y)
    elif cv_method == 'groupkfold':
        cv = StratifiedGroupKFold(n_splits=5, shuffle=False, random_state=RANDOM_STATE).split(X, y, groups)
    else:         
        raise ValueError("cv_method must be 'louo' or 'kfold'")

    res = []
    feature_importances_list = []

    for train_index, test_index in cv:
    # for train_index, test_index in tqdm(logo.split(X, y, groups), total=logo.get_n_splits(X, y, groups), desc="LOSO"):
        # print("X shape", X.shape)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        if cv_method != 'kfold':
            uid_test = groups[test_index]

        # Impute missing values using KNNImputer
        X_train.columns = X_train.columns.astype(str)
        numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns

        # Apply KNNImputer only to numeric columns 
        knn_imputer = KNNImputer(n_neighbors=3)
        X_train_imputed = knn_imputer.fit_transform(X_train[numeric_cols])
        X_test_imputed = knn_imputer.transform(X_test[numeric_cols])

        # Replace the numeric columns in the original df with imputed values
        X_train[numeric_cols] = X_train_imputed
        X_test[numeric_cols] = X_test_imputed
  
        if zero:
            # Remove any features that have the same value in all rows
            selector = VarianceThreshold(threshold=0)
            X_train_var_threshold = selector.fit_transform(X_train)
            selected_features = selector.get_support(indices=True)
            selected_columns = X_train.columns[selected_features]

            X_train = pd.DataFrame(X_train_var_threshold, columns=selected_columns)
            X_test = X_test.iloc[:, selected_features]


        if high_pariwise:
            # Identify and drop highly correlated features on the training set
            # X_train, to_drop = filter_high_correlation(X_train, threshold=correlation_threshold)
            to_drop = filter_high_correlation(X_train, threshold=correlation_threshold)
            X_train = X_train.drop(columns=to_drop)
            X_test = X_test.drop(columns=to_drop)

        if lasso:
            selected_features = select_features_lasso(X_train, y_train)
            X_train = X_train[selected_features]
            X_test = X_test[selected_features]
        
        if len(selected_features) == 0:
            # Handle case with no selected features
            raise ValueError("No features were selected by Lasso. Try using a lower alpha value or check data quality.")


        if scaling == True:
            # Apply standard scaling separately to the training and testing sets
            scaler = StandardScaler()
            X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
            X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


        # Apply oversampling if enabled
        if oversample:
            smote = SMOTE(sampling_strategy=sampling_method, random_state=42)
            X_train, y_train = smote.fit_resample(X_train, y_train)

    
        print("X shape", X_train.shape)
        # Convert back to NumPy arrays for model training and evaluation
        X_train = X_train.values
        X_test = X_test.values

        # Train the provided model instance on the training set
        model_instance.fit(X_train, y_train)

        # Evaluate the model on the test set
        y_pred = model_instance.predict(X_test)
        y_pred_proba = model_instance.predict_proba(X_test)[:, 1]

        f1_weighted = f1_score(y_test, y_pred, average='weighted')
        f1_macro = f1_score(y_test, y_pred, average='macro')
        accuracy = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)

        
        # Append results for this iteration
        if cv_method == 'loso':
            res.append((set(uid_test), auc, accuracy, f1_macro))
            print("uid: ", set(uid_test), "\t auc: ", auc, "\t f1_macro: ", f1_macro, "\t accuracy: ", accuracy)
        elif cv_method == 'kfold':
            res.append((auc, accuracy, f1_macro))
            print("\t auc: ", auc, "\t f1_macro: ", f1_macro, "\t accuracy: ", accuracy)
    
    return res

In [55]:
def personalization_fold(data, label, model_instance, zero, high_pariwise, lasso, oversample=False, sampling_method=None, correlation_threshold=None):
   
    res = []

    users = sorted(data['uid'].unique())
    print(users)


    # 각 사용자에 대해 partial personalization 수행
    for target_user in users:
        
        target_user_data = data[data['uid'] == target_user]
        target_user_data = target_user_data.sort_index()
        
        X, y, _ = data_splitting(target_user_data, label)
        
        # k-fold cross validation
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    

        for train_index, test_index in cv.split(X, y):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]


            if zero:
                selector = VarianceThreshold(threshold=0)
                X_train_var_threshold = selector.fit_transform(X_train)
                selected_features = selector.get_support(indices=True)
                selected_columns = X_train.columns[selected_features]

                X_train = pd.DataFrame(X_train_var_threshold, columns=selected_columns)
                X_test = X_test.iloc[:, selected_features]

            if high_pariwise:
                to_drop = filter_high_correlation(X_train, threshold=correlation_threshold)
                X_train = X_train.drop(columns=to_drop)
                X_test = X_test.drop(columns=to_drop)

            if lasso:
                selected_features = select_features_lasso(X_train, y_train)
                X_train = X_train[selected_features]
                X_test = X_test[selected_features]

            scaler = StandardScaler()
            X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
            # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
            X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
            
            # 이웃 수 결정
            minority_class_size = get_minority_class_size(y_train)
            n_neighbors = max(1, min(5, minority_class_size - 1))  # 최소값을 1로 설정하여 0이 되지 않도록 함
            
            if oversample:
                # smote = SMOTE(sampling_strategy=sampling_method, random_state=42)
                # X_train, y_train = smote.fit_resample(X_train, y_train)
                
                if minority_class_size > 1:  # 소수 클래스 샘플이 1보다 많을 때만 SMOTE 적용
                    smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=n_neighbors)
                    X_train, y_train = smote.fit_resample(X_train, y_train)
                else:
                    X_train, y_train = X_train, y_train  # SMOTE를 적용할 수 없는 경우 원본 데이터 사용
  
            # 결과 확인
            # print("원래 훈련 세트 크기:", X_train.shape, y_train.shape)
            # print("SMOTE 적용 후 훈련 세트 크기:", X_train_resampled.shape, y_train_resampled.shape)

            X_train = X_train.values
            X_test = X_test.values

            model_instance.fit(X_train, y_train)

    
            y_pred = model_instance.predict(X_test)
            y_pred_proba = model_instance.predict_proba(X_test)[:, 1]

            f1_macro = f1_score(y_test, y_pred, average='macro')
            accuracy = accuracy_score(y_test, y_pred)
            balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
           
            res.append((str(target_user), auc, accuracy, balanced_accuracy, f1_macro))
            print("uid: ", target_user, "\t auc: ", auc, "\t accuracy: ", accuracy,  "\t balanced_accuray", balanced_accuracy, "\t f1_macro: ", f1_macro, )
         
            # ROC AUC 점수 계산 (두 개의 클래스가 있는 경우에만)
            if len(np.unique(y_test)) <= 1 and y_pred_proba is None:
                print("AUC 점수를 계산할 수 없습니다. y_test에 하나의 클래스만 존재합니다.")



    return res

In [ ]:
# label_list = ['phq2_result_binary', 'gad2_result_binary', 'stress_result_binary', 'posNeg_result_binary', 'arousal_result_binary']
label_list = [ 'gad2_result_binary']

correlation_threshold_list = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 2.0]
zero_variance = True
high_pariwise = True
oversample = True
lasso = False
scaling = False
cv_method = 'loso'
shuffle = True
n_trials = 100
k = 5

for label in label_list:
    print(f"Processing label: {label}")
    
    # 모델 결과를 저장할 데이터프레임 초기화
    results = pd.DataFrame()
    
    for correlation_threshold in correlation_threshold_list:
        print(f"Threshold: {correlation_threshold}")

        df = merged_df.copy()
               
        # Identify groups with only one class label
        # single_class_groups = check_single_class_groups(df, label)
        user_with_skewed_label = remove_user_with_skewed_label(df, label)

        # user_with_skewed_label에 2, 14, 16 인덱스 추가
        user_with_skewed_label = list(user_with_skewed_label)
        index_list = []
        # index_list = [2, 14, 16]
        for idx in index_list:
            if idx not in user_with_skewed_label:
                user_with_skewed_label.append(idx)
        user_with_skewed_label = pd.Index(user_with_skewed_label)

        print(f"Users with skewed label: {user_with_skewed_label}")
        df = df[~df['uid'].isin(user_with_skewed_label)]
        # print(df.shape)
    
 
        
        MODELS = [
                      ('Decision Tree', DecisionTreeClassifier(random_state=42)),
                      ('Random Forest', RandomForestClassifier(random_state=42)),
                      ('AdaBoost', AdaBoostClassifier(random_state=42)),
                      ('XGBoost', XGBClassifier(random_state=42)),
                      ('LDA', LinearDiscriminantAnalysis()),
                      ('kNN', KNeighborsClassifier()),
                      ('SVM', SVC(probability=True, random_state=42)),
                    ]


        for model_name, model in tqdm(MODELS, desc="Processing Models"):

            print(f"Running {model_name}") 
            
            # Generalization
            res = train_ml(df, label, model, zero=zero_variance, high_pariwise=high_pariwise, lasso=lasso, scaling=scaling,
                cv_method = cv_method, 
                oversample=oversample,
                sampling_method='auto', correlation_threshold=correlation_threshold)

            
            if cv_method == 'loso':
                table = pd.DataFrame(res, columns=['uid', 'auc', 'accuracy', 'f1_macro'])
            elif cv_method == 'kfold':  
                table = pd.DataFrame(res, columns=['auc', 'accuracy', 'f1_macro', 'f1_weighted'])        

            # Personalization
            res = personalization_fold(df, label, model, zero=zero_variance, high_pariwise=high_pariwise, lasso=lasso,
                oversample=oversample,
                sampling_method='auto', correlation_threshold = correlation_threshold)
    
    
            table['model'] = model_name
            table['label'] = label
            table['correlation_threshold'] = correlation_threshold  

            print(table)


            average_metrics = table.mean(numeric_only=True)
            std_metrics = table.std(numeric_only=True)


           # Create a new row with both average and standard deviation
            new_row = {
                'uid': 'Average',
                'auc': average_metrics['auc'],
                'auc_std': std_metrics['auc'],
                'accuracy': average_metrics['accuracy'],
                'accuracy_std': std_metrics['accuracy'],
                # 'balanced_accuracy': average_metrics['balanced_accuracy'],
                # 'balanced_accuracy_std': std_metrics['balanced_accuracy'],
                'f1_macro': average_metrics['f1_macro'],
                'f1_macro_std': std_metrics['f1_macro'],
                'model': model_name,
                'label': label,
                'correlation_threshold': correlation_threshold
            }

            
            new_row_df = pd.DataFrame([new_row])

            # 기존 DataFrame에 행 추가
            results = pd.concat([results, table, new_row_df], ignore_index=True)


            print("Average Metrics:")
            print(average_metrics)
            print("-"*50)



        # 모든 모델의 결과를 CSV 파일로 저장
        results.to_csv("RESULT/" + label + "/LOSO/" + "all_data.csv", index=False)
        print("File saved successfully")

Processing label: phq2_result_binary
Threshold: 0.7
Users with skewed label: Index([4, 8, 10, 12, 13, 14, 15, 19, 20], dtype='int64')


Processing Models:   0%|          | 0/7 [00:00<?, ?it/s]

Running Decision Tree
X shape (590, 987)
uid:  {1} 	 auc:  0.4743589743589744 	 f1_macro:  0.45806451612903226 	 accuracy:  0.4642857142857143
X shape (576, 968)
uid:  {2} 	 auc:  0.5397727272727273 	 f1_macro:  0.4857142857142857 	 accuracy:  0.5
X shape (602, 964)
uid:  {3} 	 auc:  0.45691609977324266 	 f1_macro:  0.4012654012654012 	 accuracy:  0.46551724137931033
X shape (580, 963)
uid:  {5} 	 auc:  0.4916666666666667 	 f1_macro:  0.4909862142099682 	 accuracy:  0.53125
X shape (580, 956)
uid:  {6} 	 auc:  0.43333333333333335 	 f1_macro:  0.35714285714285715 	 accuracy:  0.3888888888888889
X shape (538, 962)
uid:  {7} 	 auc:  0.42407407407407405 	 f1_macro:  0.4110305958132045 	 accuracy:  0.5064935064935064
X shape (552, 964)
uid:  {9} 	 auc:  0.5630952380952381 	 f1_macro:  0.5262187088274045 	 accuracy:  0.532258064516129
X shape (528, 954)
uid:  {11} 	 auc:  0.5678228021978022 	 f1_macro:  0.5416149068322982 	 accuracy:  0.5853658536585366
X shape (604, 972)
uid:  {16} 	 auc:  

Processing Models:  14%|█▍        | 1/7 [01:14<07:25, 74.19s/it]

X shape (604, 977)
uid:  {18} 	 auc:  0.41666666666666663 	 f1_macro:  0.4097222222222222 	 accuracy:  0.4117647058823529
     uid       auc  accuracy  f1_macro          model               label  \
0    {1}  0.474359  0.464286  0.458065  Decision Tree  phq2_result_binary   
1    {2}  0.539773  0.500000  0.485714  Decision Tree  phq2_result_binary   
2    {3}  0.456916  0.465517  0.401265  Decision Tree  phq2_result_binary   
3    {5}  0.491667  0.531250  0.490986  Decision Tree  phq2_result_binary   
4    {6}  0.433333  0.388889  0.357143  Decision Tree  phq2_result_binary   
5    {7}  0.424074  0.506494  0.411031  Decision Tree  phq2_result_binary   
6    {9}  0.563095  0.532258  0.526219  Decision Tree  phq2_result_binary   
7   {11}  0.567823  0.585366  0.541615  Decision Tree  phq2_result_binary   
8   {16}  0.715278  0.653846  0.649175  Decision Tree  phq2_result_binary   
9   {17}  0.614706  0.675214  0.542593  Decision Tree  phq2_result_binary   
10  {18}  0.416667  0.411765  0

...

uid:  {18} 	 auc:  0.6285714285714286 	 f1_macro:  0.5096153846153846 	 accuracy:  0.6470588235294118
     uid       auc  accuracy  f1_macro    model               label  \
0    {1}  0.336735  0.500000  0.455556  XGBoost  gad2_result_binary   
1    {2}  0.425000  0.500000  0.494949  XGBoost  gad2_result_binary   
2    {3}  0.420000  0.206897  0.206897  XGBoost  gad2_result_binary   
3    {4}  0.419815  0.710345  0.437361  XGBoost  gad2_result_binary   
4    {5}  0.670588  0.625000  0.623529  XGBoost  gad2_result_binary   
5    {6}  0.304688  0.444444  0.345455  XGBoost  gad2_result_binary   
6    {7}  0.424815  0.597403  0.484783  XGBoost  gad2_result_binary   
7    {8}  0.416667  0.779221  0.437956  XGBoost  gad2_result_binary   
8    {9}  0.578892  0.467742  0.443265  XGBoost  gad2_result_binary   
9   {10}  0.624183  0.735849  0.660256  XGBoost  gad2_result_binary   
10  {11}  0.704360  0.626016  0.573946  XGBoost  gad2_result_binary   
11  {16}  0.806061  0.576923  0.571214  XGBoos

Processing Models:  71%|███████▏  | 5/7 [12:05<04:48, 144.44s/it]

X shape (1020, 1019)
uid:  {18} 	 auc:  0.7071428571428571 	 f1_macro:  0.5641025641025641 	 accuracy:  0.5882352941176471
     uid       auc  accuracy  f1_macro model               label  \
0    {1}  0.497449  0.500000  0.333333   LDA  gad2_result_binary   
1    {2}  0.515000  0.533333  0.497608   LDA  gad2_result_binary   
2    {3}  0.280000  0.586207  0.369565   LDA  gad2_result_binary   
3    {4}  0.329829  0.475862  0.380063   LDA  gad2_result_binary   
4    {5}  0.680392  0.625000  0.611336   LDA  gad2_result_binary   
5    {6}  0.437500  0.583333  0.422460   LDA  gad2_result_binary   
6    {7}  0.516296  0.454545  0.454453   LDA  gad2_result_binary   
7    {8}  0.433877  0.246753  0.236320   LDA  gad2_result_binary   
8    {9}  0.568443  0.580645  0.580208   LDA  gad2_result_binary   
9   {10}  0.650327  0.584906  0.583571   LDA  gad2_result_binary   
10  {11}  0.562500  0.487805  0.486583   LDA  gad2_result_binary   
11  {16}  0.621212  0.653846  0.626794   LDA  gad2_result_bin

Processing Models:  86%|████████▌ | 6/7 [14:16<02:19, 139.86s/it]

X shape (1020, 1019)
uid:  {18} 	 auc:  0.48571428571428565 	 f1_macro:  0.5296442687747036 	 accuracy:  0.5882352941176471
     uid       auc  accuracy  f1_macro model               label  \
0    {1}  0.571429  0.607143  0.606641   kNN  gad2_result_binary   
1    {2}  0.425000  0.533333  0.444444   kNN  gad2_result_binary   
2    {3}  0.667500  0.637931  0.517241   kNN  gad2_result_binary   
3    {4}  0.553635  0.544828  0.474638   kNN  gad2_result_binary   
4    {5}  0.562745  0.625000  0.611336   kNN  gad2_result_binary   
5    {6}  0.222656  0.361111  0.295319   kNN  gad2_result_binary   
6    {7}  0.578519  0.584416  0.572222   kNN  gad2_result_binary   
7    {8}  0.501812  0.545455  0.450561   kNN  gad2_result_binary   
8    {9}  0.622780  0.580645  0.578892   kNN  gad2_result_binary   
9   {10}  0.700980  0.660377  0.645089   kNN  gad2_result_binary   
10  {11}  0.663808  0.650407  0.617653   kNN  gad2_result_binary   
11  {16}  0.563636  0.576923  0.576296   kNN  gad2_result_bi